In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, log_loss
from torchsummary import summary
from sklearn.preprocessing import MinMaxScaler
import torch
import matplotlib.pyplot as plt
import os 
os.chdir("D:/Training/Academy/ML(Python)/Datasets")
from sklearn.preprocessing import LabelEncoder 

In [2]:
brupt = pd.read_excel("Bankruptcy.xlsx", sheet_name="data", index_col=0)
X = brupt.drop(['D', 'YR'], axis=1).values
y = brupt['D'].values

In [3]:
X.shape

(132, 24)

In [4]:
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=25,stratify=y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 

In [5]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([92, 24])
torch.Size([92])


### Model Definition

In [6]:
torch.manual_seed(25)
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=1))

In [7]:
summary(model, (1,X_scl_trn.shape[1]))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 1]              25
Total params: 25
Trainable params: 25
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [8]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

Generating predictions with default weights

In [9]:
y_pred = model(X_torch.float())
y_pred[:3]

tensor([[-0.1047],
        [-0.0804],
        [ 0.0306]], grad_fn=<SliceBackward0>)

In [10]:
print(y_torch.shape)

torch.Size([92])


In [11]:
y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([92, 1])
torch.Size([92, 1])


### Initial Log Loss

In [12]:
loss = criterion(y_pred, y_torch.float())
loss

tensor(0.6889, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [13]:
for epoch in np.arange(0,1000):
       # Forward pass: Compute predicted y by passing x to the model
       y_pred_prob = model(X_torch.float())

       # Compute and print loss
       loss = criterion(y_pred_prob, y_torch.float())
       if epoch%100 == 0:
          print('epoch: ', epoch+1,' loss: ', loss.item())

       # Zero gradients, perform a backward pass, and update the weights.
       optimizer.zero_grad()

       # perform a backward pass (backpropagation)
       loss.backward()

       # Update the parameters
       optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.6889256834983826
epoch:  101  loss:  0.6296066641807556
epoch:  201  loss:  0.5907428860664368
epoch:  301  loss:  0.5624328255653381
epoch:  401  loss:  0.5407347679138184
epoch:  501  loss:  0.5234107971191406
epoch:  601  loss:  0.5091308355331421
epoch:  701  loss:  0.49706318974494934
epoch:  801  loss:  0.4866637885570526
epoch:  901  loss:  0.4775616228580475


### Training Set Log Loss

In [23]:
loss

tensor(0.4696, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [15]:
X_torch_test = torch.from_numpy(X_scl_tst)
type(X_torch_test)

torch.Tensor

In [16]:
lin_output = model(X_torch_test.float()) # Equivalent predict_proba / predict
type(lin_output)

torch.Tensor

In [17]:
np_out = lin_output.detach().numpy()
y_pred_prob = 1 / (1 + np.exp(-np_out))
y_pred_prob[:5]

array([[3.2044971e-01],
       [6.5234900e-01],
       [3.3033293e-01],
       [1.2736614e-05],
       [6.2609571e-01]], dtype=float32)

`detach().numpy()` converts `torch.Tensor` into `numpy` object

In [18]:
y_pred_prob.shape

(40, 1)

In [19]:
y_pred_prob = y_pred_prob.reshape(y_test.shape[0],)
y_pred_prob.shape

(40,)

In [20]:
y_pred = np.where(y_pred_prob >= 0.5,1,0)
y_pred[:5]

array([0, 1, 0, 0, 1])

### Test Set Accuracy Score

In [21]:
print(accuracy_score(y_test,y_pred))

0.7


### Test Set Log Loss

In [22]:
log_loss(y_test, y_pred_prob)

0.5643851023610289